# 📝 Garment Generation with Stable Diffusion

This notebook demonstrates how to generate fashion garments using Stable Diffusion, a state-of-the-art text-to-image generation model. We'll create a simple pipeline for generating various clothing items from text descriptions.

## 1. Imports and Dependencies

Loading all necessary libraries for image generation, visualization, and deep learning operations.

In [ ]:
import torch
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
import warnings
warnings.filterwarnings('ignore')

## 2. Simple Configuration Class

Setting up device configuration and model parameters for consistent garment generation across different hardware setups.

In [ ]:
class SimpleConfig:
    """Simple configuration for garment generation"""
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    MODEL_NAME = "runwayml/stable-diffusion-v1-5"  # Free, reliable model
    IMAGE_SIZE = 512
    SEED = 42

print(f"🖥️ Using device: {SimpleConfig.DEVICE}")

## 3. Simple Garment Generator Class

Main class that handles the Stable Diffusion pipeline initialization and garment generation from text prompts.

In [ ]:
class SimpleGarmentGenerator:
    """Simple garment generator using Stable Diffusion"""
    
    def __init__(self):
        print("🔄 Loading Stable Diffusion model...")
        
        # Load the pre-trained model
        self.pipe = StableDiffusionPipeline.from_pretrained(
            SimpleConfig.MODEL_NAME,
            torch_dtype=torch.float16 if SimpleConfig.DEVICE == "cuda" else torch.float32,
            safety_checker=None,
            requires_safety_checker=False
        )
        
        # Move to device
        self.pipe = self.pipe.to(SimpleConfig.DEVICE)
        
        # Enable memory efficient settings
        if SimpleConfig.DEVICE == "cuda":
            self.pipe.enable_attention_slicing()
        
        print("✅ Model loaded successfully!")

## 4. Outfit Generation Method

The core method that takes text prompts and generates fashion images using enhanced prompts and negative prompts for better results.

In [ ]:
def generate_outfit(self, prompt, seed=None):
    """Generate outfit from text prompt"""
    
    # Set seed for reproducible results
    if seed is None:
        seed = SimpleConfig.SEED
    
    generator = torch.Generator(device=SimpleConfig.DEVICE).manual_seed(seed)
    
    # Enhanced prompt for better fashion results
    fashion_prompt = f"high quality fashion photography, {prompt}, professional lighting, detailed fabric texture, fashion model, studio photography"
    
    # Negative prompt to avoid bad results
    negative_prompt = "low quality, blurry, distorted, ugly, bad anatomy, extra limbs"
    
    print(f"🎨 Generating: {prompt}")
    
    # Generate image
    with torch.autocast(SimpleConfig.DEVICE):
        result = self.pipe(
            prompt=fashion_prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=20,  # Fast generation
            guidance_scale=7.5,
            generator=generator,
            height=SimpleConfig.IMAGE_SIZE,
            width=SimpleConfig.IMAGE_SIZE
        ).images[0]
    
    return result

## 5. Helper Functions

Utility functions for displaying generated images and saving results to disk with proper naming conventions.

In [ ]:
def display_results(prompt, generated_image):
    """Display the generated result"""
    plt.figure(figsize=(8, 6))
    plt.imshow(generated_image)
    plt.title(f"Generated Outfit: {prompt}", fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def save_result(image, prompt, filename=None):
    """Save generated image"""
    if filename is None:
        # Create safe filename from prompt
        safe_prompt = "".join(c for c in prompt if c.isalnum() or c in (' ', '-', '_')).rstrip()
        filename = f"outfit_{safe_prompt.replace(' ', '_')}.png"
    
    image.save(filename)
    print(f"💾 Saved: {filename}")

## 6. Main Testing Function

Comprehensive test function that generates outfits from predefined prompts and handles the complete pipeline from generation to saving results.

In [ ]:
def test_garment_generation():
    """Simple test with two text prompts"""
    
    print("=" * 60)
    print("🎭 SIMPLE GARMENT GENERATION TEST")
    print("=" * 60)
    
    # Initialize generator
    generator = SimpleGarmentGenerator()
    
    # Test prompts
    test_prompts = [
        "elegant red evening dress",
        "casual blue jeans with white t-shirt"
    ]
    
    print(f"\n🧪 Testing with {len(test_prompts)} prompts:")
    
    results = []
    
    # Generate outfits for each prompt
    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n--- Test {i}/{len(test_prompts)} ---")
        
        try:
            # Generate image
            generated_image = generator.generate_outfit(prompt)
            
            # Display result
            display_results(prompt, generated_image)
            
            # Save result
            save_result(generated_image, prompt)
            
            results.append({
                'prompt': prompt,
                'image': generated_image
            })
            
            print(f"✅ Successfully generated: {prompt}")
            
        except Exception as e:
            print(f"❌ Error generating '{prompt}': {e}")
    
    print(f"\n{'=' * 60}")
    print("🎉 TESTING COMPLETE!")
    print(f"✅ Generated {len(results)}/{len(test_prompts)} outfits successfully")
    print("=" * 60)
    
    return results

## 7. Interactive Custom Generation Function

Function for generating custom outfits with user-provided prompts, including error handling and result management.

In [ ]:
def generate_custom_outfit(generator, custom_prompt):
    """Generate outfit with custom prompt"""
    print(f"\n🎨 Generating custom outfit...")
    
    try:
        generated_image = generator.generate_outfit(custom_prompt)
        display_results(custom_prompt, generated_image)
        save_result(generated_image, custom_prompt)
        print(f"✅ Successfully generated: {custom_prompt}")
        return generated_image
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

## 8. Main Execution Pipeline

Running the complete garment generation pipeline with the predefined test cases to demonstrate the system's capabilities.

In [ ]:
print("🚀 Starting Simple Garment Generation Pipeline")
results = test_garment_generation()